In [ ]:
%gui asyncio
from progressivis_nb_widgets.nbwidgets import *

In [ ]:
"""
Clustering datasets may be found at
https://cs.joensuu.fi/sipu/datasets/
"""
from progressivis import Scheduler, Every#, log_level
from progressivis.cluster import MBKMeans
from progressivis.io import CSVLoader
from progressivis.vis import MCScatterPlot
from progressivis.datasets import get_dataset
from progressivis.stats import RandomTable

try:
    s = scheduler
except NameError:
    s = Scheduler()
    #log_level(package="progressivis.cluster")

data = CSVLoader(get_dataset('cluster:s1'),sep='\\s+',skipinitialspace=True,header=None,index_col=False,scheduler=s)
mbkmeans = MBKMeans(columns=['_0', '_1'], n_clusters=15, batch_size=100, is_input=False, scheduler=s)
sp = MCScatterPlot(scheduler=s, classes=[('Scatterplot', '_0', '_1', mbkmeans)], approximate=True)
sp.create_dependent_modules(data,'table')
mbkmeans.input.table = sp['Scatterplot'].range_query_2d.output.table
mbkmeans.create_dependent_modules()
prn = Every(scheduler=s)
prn.input.df = mbkmeans.output.table

sp.move_point = mbkmeans.moved_center # for input management
# Create the dashboard object
psboard = PsBoard(s)
# Visualisations require registration :
# 1) When widget provides the link_module() method do :
psboard.register_visualisation(Scatterplot(), sp)
# 2) When widget is generic or for overloading the link_module() method do:
# psboard.register_visualisation(FooWidget(), bar_module, glue=glue_func)
# NB: glue_func(widget, module)-> [coroutine] must be provided
# Start application :
aio.create_task(s.start(coros=psboard.coroutines));

In [ ]:
import time
time.sleep(3) # for voilà ...

In [ ]:
display(psboard)